# OTX API DEMO (MMPretrain Example)

## Customization Training API

Select a framework & import adapter modules.

We'll choose MMpretrain here.

"""
Environment:
- mmpretrain-1.0.0rc8
- mmcv-2.0.1
- mmengine-0.7.4
- mmdeploy-1.2.0

"""

## Prepare Dataset & DataLoader
1. Prepare a dataset and enter path into Dataset

    - Convert to OTX's DatasetEntity and Label Schema by leveraging Datumaro's features through paths (path -> Datumaro -> OTX DatasetEntity & LabelSchema)

In [1]:
from otx.v2.adapters.torch.mmengine.mmpretrain import Dataset
dataset = Dataset(
    train_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/classification_dataset_class_incremental",
    val_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/classification_dataset_class_incremental",
    test_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/classification_dataset_class_incremental",
)

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/openvino/pyopenvino/__init__.py:10: FutureWarning: The module is private and following namespace `pyopenvino` will be removed in the future
  warnings.warn(message="The module is private and following namespace " "`pyopenvino` will be removed in the future", category=FutureWarning)


2-1. Build Torch Dataset from MMCV config (filepath or dict) -> torch.utils.data.Dataset

    - User can build a dataset from a config file or dictionary.

In [2]:
train_dataloader = dataset.train_dataloader()
print(f"Dataset type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")
print(f"Number of classes: {dataset.num_classes}")

[*] Detected dataset format: imagenet
[*] Detected task type: CLASSIFICATION
2023-07-10 15:41:10,743 | INFO : Try to create a 0 size memory pool.
Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 32
Dataset size: 32
Number of classes: 3


In [3]:
# Customize batch_size
train_dataloader = dataset.train_dataloader(batch_size=4)
print(f"DataLoader type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 8
Dataset size: 32


In [4]:
val_dataloader = dataset.val_dataloader()
print(f"DataLoader type: {type(val_dataloader)}")
print(f"Length of DataLoader: {len(val_dataloader)}")
print(f"Dataset size: {len(val_dataloader.dataset)}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 32
Dataset size: 32


In [5]:
test_dataloader = dataset.test_dataloader()
print(f"DataLoader type: {type(test_dataloader)}")
print(f"Length of DataLoader: {len(test_dataloader)}")
print(f"Dataset size: {len(test_dataloader.dataset)}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 32
Dataset size: 32


## Prepare Model
### Model provided by OTX

In [6]:
## OTX Custom Model
from otx.v2.adapters.torch.mmengine.mmpretrain import get_model
otx_model = get_model(
    config="/home/harimkan/workspace/repo/otx-fork-3/src/otx/v2/configs/classification/otx_efficientnet_b0.yaml",
    num_classes=dataset.num_classes
)
print(f"Model type: {type(otx_model)}")

2023-07-10 15:41:22,859 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-07-10 15:41:22,883 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-07-10 15:41:22,947 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
Model type: <class 'otx.v2.adapters.torch.mmengine.mmpretrain.modules.models.classifiers.sam_classifier.SAMImageClassifier'>


### Model provided by mmpretrain

In [7]:
# mmpretrain's pre-defined model
from mmpretrain import get_model
mmpretrain_model = get_model("resnet18_8xb32_in1k")
print(f"Model type: {type(mmpretrain_model)}")

Model type: <class 'mmpretrain.models.classifiers.image.ImageClassifier'>


## Training

Users can use each framework's training provided by OTX. (Engine)

- The engine requires the necessary models and DataLoaders for each framework.

In [16]:
from otx.v2.adapters.torch.mmengine.mmpretrain.engine import MMPTEngine

# OTX Model Training
engine = MMPTEngine()

# Training without validation
output = engine.train(
    model=otx_model,
    train_dataloader=train_dataloader,
    work_dir="/tmp/otx-test",
    max_epochs=3,
)

print(f"Output type: {type(output)}")

07/10 15:43:39 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 822569775
    GPU 0,1: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.7, V11.7.64
    GCC: gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=s

07/10 15:43:40 - mmengine - INFO - Exp name: 20230710_154339
07/10 15:43:40 - mmengine - INFO - Epoch(train) [1][8/8]  lr: 1.0000e-02  eta: 0:00:00  time: 0.0325  data_time: 0.0013  memory: 584  loss: 7.0541
07/10 15:43:40 - mmengine - INFO - Saving checkpoint at 1 epochs
07/10 15:43:40 - mmengine - WARNING - `save_param_scheduler` is True but `self.param_schedulers` is None, so skip saving parameter schedulers
07/10 15:43:40 - mmengine - INFO - Exp name: 20230710_154339
07/10 15:43:40 - mmengine - INFO - Epoch(train) [2][8/8]  lr: 1.0000e-02  eta: 0:00:00  time: 0.0335  data_time: 0.0012  memory: 584  loss: 6.4864
07/10 15:43:40 - mmengine - INFO - Saving checkpoint at 2 epochs
07/10 15:43:41 - mmengine - INFO - Exp name: 20230710_154339
07/10 15:43:41 - mmengine - INFO - Epoch(train) [3][8/8]  lr: 1.0000e-02  eta: 0:00:00  time: 0.0330  data_time: 0.0012  memory: 584  loss: 5.9475
07/10 15:43:41 - mmengine - INFO - Saving checkpoint at 3 epochs
Output type: <class 'otx.v2.adapters.to

In [17]:
val_score = engine.val(val_dataloader=val_dataloader)
print(f"Val Metric: {val_score}")

test_score = engine.test(test_dataloader=test_dataloader)
print(f"Test Metric: {test_score}")

07/10 15:43:44 - mmengine - INFO - Epoch(val) [3][32/32]    accuracy/top1: 25.0000  data_time: 0.0003  time: 0.0085
07/10 15:43:44 - mmengine - INFO - The best checkpoint with 25.0000 accuracy/top1 at 3 epoch is saved to best_accuracy_top1_epoch_3.pth.
Val Metric: {'accuracy/top1': 25.0}
07/10 15:43:45 - mmengine - INFO - Epoch(test) [32/32]    accuracy/top1: 25.0000  data_time: 0.0003  time: 0.0088
Test Metric: {'accuracy/top1': 25.0}


In [18]:
# OR
# Training with validation
trained_model = engine.train(
    model=otx_model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    work_dir="/tmp/otx-test",
    max_epochs=3,
)

07/10 15:43:46 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1104582442
    GPU 0,1: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.7, V11.7.64
    GCC: gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=

07/10 15:43:47 - mmengine - INFO - Exp name: 20230710_154346
07/10 15:43:47 - mmengine - INFO - Epoch(train) [1][8/8]  lr: 1.0000e-02  eta: 0:00:00  time: 0.0328  data_time: 0.0013  memory: 630  loss: 7.0208
07/10 15:43:47 - mmengine - INFO - Saving checkpoint at 1 epochs
07/10 15:43:47 - mmengine - WARNING - `save_param_scheduler` is True but `self.param_schedulers` is None, so skip saving parameter schedulers
07/10 15:43:47 - mmengine - INFO - Epoch(val) [1][32/32]    accuracy/top1: 25.0000  data_time: 0.0003  time: 0.0084
07/10 15:43:47 - mmengine - INFO - The best checkpoint with 25.0000 accuracy/top1 at 1 epoch is saved to best_accuracy_top1_epoch_1.pth.
07/10 15:43:47 - mmengine - INFO - Exp name: 20230710_154346
07/10 15:43:47 - mmengine - INFO - Epoch(train) [2][8/8]  lr: 1.0000e-02  eta: 0:00:00  time: 0.0341  data_time: 0.0012  memory: 630  loss: 6.3152
07/10 15:43:47 - mmengine - INFO - Saving checkpoint at 2 epochs
07/10 15:43:48 - mmengine - INFO - Epoch(val) [2][32/32]   

In [19]:
val_score = engine.val()
print(f"Val Metric: {val_score}")

test_score = engine.test(test_dataloader=test_dataloader)
print(f"Test Metric: {test_score}")

07/10 15:43:51 - mmengine - INFO - Epoch(val) [3][32/32]    accuracy/top1: 53.1250  data_time: 0.0003  time: 0.0085
Val Metric: {'accuracy/top1': 53.125}
07/10 15:43:51 - mmengine - INFO - Epoch(test) [32/32]    accuracy/top1: 53.1250  data_time: 0.0003  time: 0.0086
Test Metric: {'accuracy/top1': 53.125}


## Predict

In [20]:
sample = "/home/harimkan/workspace/repo/otx-fork-3/tests/assets/classification_dataset_class_incremental/2/22.jpg"

predict_output = engine.predict(
    model=trained_model,
    img=sample
)

print(predict_output)

[{'pred_scores': array([0.31217298, 0.32922497, 0.35860205], dtype=float32), 'pred_label': 2, 'pred_score': 0.35860204696655273}]
